In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost
import pickle

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
data_2017 = pd.read_csv("https://files.data.gouv.fr/geo-dvf/latest/csv/2017/departements/69.csv.gz")
data_2018 = pd.read_csv("https://files.data.gouv.fr/geo-dvf/latest/csv/2018/departements/69.csv.gz")
data_2019 = pd.read_csv("https://files.data.gouv.fr/geo-dvf/latest/csv/2019/departements/69.csv.gz")
data_2020 = pd.read_csv("https://files.data.gouv.fr/geo-dvf/latest/csv/2020/departements/69.csv.gz")
data_2021 = pd.read_csv("https://files.data.gouv.fr/geo-dvf/latest/csv/2021/departements/69.csv.gz")
data_2022 = pd.read_csv("https://files.data.gouv.fr/geo-dvf/latest/csv/2022/departements/69.csv.gz")

C:\Users\bastien\AppData\Local\Temp\ipykernel_7720\1170027473.py:1: DtypeWarning: Columns (14,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2017 = pd.read_csv("https://files.data.gouv.fr/geo-dvf/latest/csv/2017/departements/69.csv.gz")
C:\Users\bastien\AppData\Local\Temp\ipykernel_7720\1170027473.py:2: DtypeWarning: Columns (14,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2018 = pd.read_csv("https://files.data.gouv.fr/geo-dvf/latest/csv/2018/departements/69.csv.gz")
C:\Users\bastien\AppData\Local\Temp\ipykernel_7720\1170027473.py:3: DtypeWarning: Columns (14,16,17,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data_2019 = pd.read_csv("https://files.data.gouv.fr/geo-dvf/latest/csv/2019/departements/69.csv.gz")
C:\Users\bastien\AppData\Local\Temp\ipykernel_7720\1170027473.py:4: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_me

In [4]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df[[
        "date_mutation", 
        "valeur_fonciere", 
        "code_postal", 
        "lot1_surface_carrez",
        "lot2_surface_carrez",
        "lot3_surface_carrez",
        "lot4_surface_carrez",
        "lot5_surface_carrez",
        "nombre_lots",
        "type_local", 
        "surface_reelle_bati", 
        "nombre_pieces_principales",
        "nature_culture",
        "nature_culture_speciale",
        "surface_terrain"
    ]]
    df = df.rename({
        "date_mutation": "date",
        "valeur_fonciere": "price",
        "code_postal": "district_id",
        "lot1_surface_carrez": "lot1_surface",
        "lot2_surface_carrez": "lot2_surface",
        "lot3_surface_carrez": "lot3_surface",
        "lot4_surface_carrez": "lot4_surface",
        "lot5_surface_carrez": "lot5_surface",
        "nombre_lots": "lots",
        "type_local": "type",
        "surface_reelle_bati": "surface",
        "nombre_pieces_principales": "rooms",
        "nature_culture": "extra",
        "nature_culture_speciale": "extra_special",
        "surface_terrain": "land_surface"
    }, axis=1)

    df = df[df["district_id"].isin([69001.0, 69002.0, 69003.0, 69004.0, 69005.0, 69006.0, 69007.0, 69008.0, 69009.0])]
    df = df[
        df["price"].notna() 
        & df["type"].notna() 
        & df["rooms"].notna() 
        & df["surface"].notna()
        & (df["surface"] >= 9.0)
        & ((df["price"] > 100_000) & (df["price"] < 2_000_000))
        & (df["type"] != "Local industriel. commercial ou assimilé")
    ]
    df["date"] = pd.to_datetime(df["date"])
    df[[
        "lot1_surface",
        "lot2_surface",
        "lot3_surface",
        "lot4_surface",
        "lot5_surface",
        "lots",
        "land_surface"
    ]] = df[[
        "lot1_surface",
        "lot2_surface",
        "lot3_surface",
        "lot4_surface",
        "lot5_surface",
        "lots",
        "land_surface"
    ]].fillna(0.0)
    df["district_id"] = df["district_id"].astype(int)
    df["rooms"] = df["rooms"].astype(int)
    df["type"] = df["type"].map({"Appartement": "apartment", "Maison": "house"})
    df["extra"] = df["extra"].map({
        "sols": "ground", 
        "terrains d'agrément": "pleasure_grounds", 
        "vergers": "orchards", 
        "jardins": "gardens",
        "futaies feuillues": "deciduous_forests", 
        "taillis simples": "simple_copses"
    })
    df["extra_special"] = df["extra_special"].map({
        "Parc": "park",
        "Dépendances d'ensemble immobilier": "building_complex",
        "Jardin potager": "vegetable_garden"
    })
    df["extra"].fillna("none", inplace=True)
    df["extra_special"].fillna("no_special", inplace=True)
    return df

In [12]:
df = pd.concat([
    clean_data(data_2017),
    clean_data(data_2018),
    clean_data(data_2019),
    clean_data(data_2020),
    clean_data(data_2021),
    clean_data(data_2022)
]).reset_index()
df.drop("index", axis=1, inplace=True)
df.to_csv("./data/data.csv", index=False)

In [13]:
df = pd.read_csv("./data/data.csv")
df.drop("date", axis=1, inplace=True)

In [14]:
one_hot_type = pd.get_dummies(df["type"])
one_hot_extra = pd.get_dummies(df["extra"])
one_hot_extra_special = pd.get_dummies(df["extra_special"])
one_hot_district = pd.get_dummies(df["district_id"], prefix="district")

In [15]:
df_full = df.copy()
df_full.drop(["type", "extra", "district_id", "extra_special"], axis=1, inplace=True)
df_full = df_full.join([one_hot_type, one_hot_extra, one_hot_district, one_hot_extra_special])

In [40]:
df_no_extras = df.copy()
df_no_extras.drop(["type", "extra", "district_id", "extra_special"], axis=1, inplace=True)
df_no_extras = df_no_extras.join([one_hot_type, one_hot_district])

In [41]:
df_full_std_normalized = df.copy()
df_full_std_normalized.drop(["type", "extra", "district_id", "extra_special"], axis=1, inplace=True)
price = df_full_std_normalized.pop("price")
df_full_std_normalized = (df_full_std_normalized-df_full_std_normalized.mean()) / df_full_std_normalized.std()
df_full_std_normalized = df_full_std_normalized.join([one_hot_type, one_hot_extra, one_hot_district, one_hot_extra_special, price])

In [42]:
df_full_min_max_normalized = df.copy()
df_full_min_max_normalized.drop(["type", "extra", "district_id", "extra_special"], axis=1, inplace=True)
price = df_full_min_max_normalized.pop("price")
df_full_min_max_normalized = (df_full_min_max_normalized-df_full_min_max_normalized.min())/(df_full_min_max_normalized.max()-df_full_min_max_normalized.min())
df_full_min_max_normalized = df_full_min_max_normalized.join([one_hot_type, one_hot_extra, one_hot_district, one_hot_extra_special, price])

In [43]:
df_no_extras_std_normalized = df.copy()
df_no_extras_std_normalized.drop(["type", "extra", "district_id", "extra_special"], axis=1, inplace=True)
price = df_no_extras_std_normalized.pop("price")
df_no_extras_std_normalized = (df_no_extras_std_normalized-df_no_extras_std_normalized.mean()) / df_no_extras_std_normalized.std()
df_no_extras_std_normalized = df_no_extras_std_normalized.join([one_hot_type, one_hot_district, price])

In [44]:
df_no_extras_min_max_normalized = df.copy()
df_no_extras_min_max_normalized.drop(["type", "extra", "district_id", "extra_special"], axis=1, inplace=True)
price = df_no_extras_min_max_normalized.pop("price")
df_no_extras_min_max_normalized = (df_no_extras_min_max_normalized-df_no_extras_min_max_normalized.min())/(df_no_extras_min_max_normalized.max()-df_no_extras_min_max_normalized.min())
df_no_extras_min_max_normalized = df_no_extras_min_max_normalized.join([one_hot_type, one_hot_district, price])

In [46]:
def compare_models_datasets(dfs, models):
    for name_df, df in dfs.items():
        print(name_df)

        X = df.drop("price", axis=1)
        y = pd.DataFrame(df["price"])
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234, test_size=0.20, shuffle=True)

        for name_model, model in models.items():
            reg = model.fit(X_train, y_train.values.ravel())
            print(f"\t{name_model}: {reg.score(X_test, y_test.values.ravel())}")

In [47]:
dfs = {
    "full": df_full,
    "full std normalized": df_full_std_normalized,
    "full min max normalized": df_full_min_max_normalized,
    "no extras": df_no_extras,
    "no extras std normalized": df_no_extras_std_normalized,
    "no extras min max normalized": df_no_extras_min_max_normalized
}

In [48]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boost": GradientBoostingRegressor(),
    "XGBoost": xgboost.XGBRegressor(),
    # "SVR": SVR()
}

In [49]:
compare_models_datasets(dfs, models)

full
	Linear Regression: 0.5183971995458607
	Random Forest: 0.6937330422272012
	Gradient Boost: 0.6135435448945832
	XGBoost: 0.6931294022364838
full std normalized
	Linear Regression: 0.5184692643842392
	Random Forest: 0.6942277589510111
	Gradient Boost: 0.6134558049024503
	XGBoost: 0.693392202828501
full min max normalized
	Linear Regression: 0.5182865507539138
	Random Forest: 0.6937690881229563
	Gradient Boost: 0.6134391141025526
	XGBoost: 0.6929557563337277
no extras
	Linear Regression: 0.33487059717638135
	Random Forest: 0.6934833824939974
	Gradient Boost: 0.6157350565756807
	XGBoost: 0.695812018302934
no extras std normalized
	Linear Regression: 0.33485827666863544
	Random Forest: 0.6927112016334888
	Gradient Boost: 0.6156071898566499
	XGBoost: 0.6957376589000761
no extras min max normalized
	Linear Regression: 0.33500734328658655
	Random Forest: 0.6955897383320304
	Gradient Boost: 0.6156299964662381
	XGBoost: 0.6959826972049385


In [17]:
X = df_full.drop("price", axis=1)
y = pd.DataFrame(df_full["price"])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1234, test_size=0.20, shuffle=True)

In [297]:
forest = RandomForestRegressor()
param_grid = {
    "n_estimators": [100, 200, 300],
    "min_samples_split": [2, 4],
    "max_depth": [None, 4, 8]
}

grid_search = GridSearchCV(forest, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score=True)

grid_search.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 4, 8], 'min_samples_split': [2, 4],
                         'n_estimators': [100, 200, 300]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [298]:
grid_search.best_estimator_.fit

RandomForestRegressor(min_samples_split=4, n_estimators=300)

In [299]:
grid_search.best_estimator_.score(X_test, y_test)

0.7017995346861381

In [300]:
forest = RandomForestRegressor()
param_grid = {
    "n_estimators": [250, 300, 350],
    "min_samples_split": [3, 4, 5],
    "max_depth": [None, 1, 2]
}

grid_search = GridSearchCV(forest, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score=True)

grid_search.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 1, 2],
                         'min_samples_split': [3, 4, 5],
                         'n_estimators': [250, 300, 350]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [301]:
grid_search.best_estimator_

RandomForestRegressor(min_samples_split=5, n_estimators=300)

In [302]:
grid_search.best_estimator_.score(X_test, y_test)

0.7032301367582408

In [18]:
forest = RandomForestRegressor()
param_grid = {
    "n_estimators": [300],
    "min_samples_split": [5, 6, 7]
}

grid_search = GridSearchCV(forest, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error", return_train_score=True)

grid_search.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'min_samples_split': [7], 'n_estimators': [300]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [19]:
grid_search.best_estimator_

RandomForestRegressor(min_samples_split=7, n_estimators=300)

In [20]:
grid_search.best_estimator_.score(X_test, y_test)

0.7029457142412401

In [21]:
with open("model.pickle", "wb") as f:
    pickle.dump(grid_search.best_estimator_, f)

In [48]:
with open("model.pickle", "rb") as f:
    reg = pickle.load(f)